<a href="https://colab.research.google.com/github/sheryps/calculations/blob/main/mask_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K





In [ ]:
class LeNet:

	def build(width, height, depth, classes):
		# initialize the model
		model = Sequential()
		inputShape = (height, width, depth)

		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)

		# first set of CONV => RELU => POOL layers
		model.add(Conv2D(20, (5, 5), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# second set of CONV => RELU => POOL layers
		model.add(Conv2D(50, (5, 5), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(500))
		model.add(Activation("relu"))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model



In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
#from lenet import LeNet
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os















In [ ]:
# construct the argument parse and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-d", "--dataset", required=True,help="path to input dataset")
#ap.add_argument("-m", "--model", required=True,help="path to output model")
#ap.add_argument("-p", "--plot", type=str, default="plot.png",help="path to output loss/accuracy plot")
#args = vars(ap.parse_args())

# initialize the number of epochs to train for, initia learning rate,
# and batch size
EPOCHS = 15
INIT_LR = 1e-2
BS = 32

# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images('/content/drive/MyDrive/DEEP LEARNING/MASK DETECTION/mask data')))
print(len(imagePaths))
random.seed(42)
random.shuffle(imagePaths)    

In [ ]:
# loop over the input images
for imagePath in imagePaths:
  image = cv2.imread(imagePath)
  image = cv2.resize(image, (28, 28))
  image = img_to_array(image)
  data.append(image)
  label = imagePath.split(os.path.sep)[-2]
  if label=="0":
    label= 0
        
  elif label=="1":
	  label= 1
        
  elif label=="2":
	  label=2
	

#  # elif label=="3":
# #	  label=3
        
# #  elif label=="4":
# #	  label=4
        
	
# #  elif label=="5":
# #	  label=5
        
# #   elif label=="6":
# # 	  label= 6
        
# #   elif label=="7":
# # 	  label= 7
        
# #   elif label=="8":
# # 	  label= 8
        
# #   else
# # 	  label= 9 
  labels.append(label)
# ############################

IndentationError: ignored

In [ ]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(label)


# # partition the data into training and testing splits using 75% of
# # the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.35 ,random_state=42)

# # convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

TypeError: ignored

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=6)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

In [ ]:
# save the model to disk
print("[INFO] serializing network...")
model.save('/content/drive/MyDrive/DEEP LEARNING/OBJECT DETECTION/realtime1.h5')